In [1]:
from GOlabels.scripts import labels
from GOlabels.scripts import hierarchyModule
from GOlabels.scripts import associationModule

## Playing with the GO labels scripts from r/BCB
They are very straight forward and easy to use. I just need to convert the MU
dataset into the correct format to be processed.

In [2]:
hm = hierarchyModule.hierarchyModule("GOlabels/data/ontology/go-basic.obo", 0)

In [3]:
am = associationModule.associationModule("GOlabels/data/associations/ensembl_peptide_id.associations")

In [4]:
testGene = am.labeledGenes[0]

In [5]:
testGene

'ENSP00000420919'

In [6]:
testLabel = list(am.labels(testGene))[0]

In [7]:
testLabel

'GO:0044459'

In [8]:
hm.translate('GO:0009987')

'GO:0009987'

In [9]:
hm.levelOf('GO:0009987')

1

## Re-formatting the MU dataset

In [2]:
fp_gene_list = "data/networks/human/human_string_genes.txt"
file_gene_list = open(fp_gene_list, 'r')

# reads in the list of string genes
string_genes = []
for gene_line in file_gene_list.readlines():
    gene = gene_line.split()[0]
    string_genes.append(gene)
file_gene_list.close()

In [29]:
# we need the map because the go and string indexing is not consistent.
idx = 1
string_map = {}
for gene in string_genes:
    string_map[gene] = idx
    idx += 1

In [3]:
len(string_genes)

18362

In [4]:
fp_gogene_list = "data/annotations/human/go_human_ref_genes.txt"
file_gogene_list = open(fp_gogene_list, 'r')

# reads the list of GO genes in
go_genes = []
for gene_line in file_gogene_list.readlines():
    gene = gene_line.split()[0]
    go_genes.append(gene)
file_gogene_list.close()

In [7]:
filt = set()
for i in range(len(string_genes)):
    gene = string_genes[i]
    if gene in go_genes:
        filt.add(gene)

11723


In [8]:
# reads in the list of GO terms being used
fp_go_term_list = "data/annotations/human/go_human_ref_bp_terms.txt"
file_go_term_list = open(fp_go_term_list, 'r')

# reads in the list of GO terms
go_terms = []
for term_line in file_go_term_list.readlines():
    term = term_line.split()[0]
    go_terms.append(term)
file_go_term_list.close()

In [9]:
# creates dict with the gene indices as keys and the list of their go term indices as values
fp_go_adj = "data/annotations/human/go_human_ref_bp_adjacency.txt"
file_go_adj = open(fp_go_adj, 'r')

gene_anno = {}
for i in range(len(go_genes)):
    gene_anno[i+1] = {'orig': []}

for map_line in file_go_adj.readlines():
    [gene_index, term_index] = map_line.split()
    gene_index = int(gene_index)
    term_index = int(term_index)

    # uses the gene indices
    gene_anno[gene_index]['orig'] += [term_index]

file_go_adj.close()

In [10]:
# map from index back to original gene name
gene_map = {}
for i in range(len(go_genes)):
    gene = go_genes[i]
    gene_map[gene] = i + 1

In [11]:
# reads in the mapping from go terms to their indices
fp_go_term_map = "data/annotations/human/graph/go_bp.map"
file_go_term_map = open(fp_go_term_map, 'r')

go_term_map = [None]
for map_line in file_go_term_map.readlines():
    [term, index] = map_line.split()
    index = int(index)
    
    # the index is sequential and this means we've seen it before
    if len(go_term_map) == index + 1:
        go_term_map[index] += [term]
    # this means its the first occurrence
    else:
        go_term_map += [[term]]
        

file_go_term_map.close()

In [12]:
# creates a DAG of the GO terms
fp_go_links = "data/annotations/human/graph/go_bp.links"
file_go_links = open(fp_go_links, 'r')

index_tree = {}
for link_line in file_go_links.readlines():
    [parent_index, child_index] = link_line.split()
    parent_index = int(parent_index)
    child_index = int(child_index)

    if not parent_index in index_tree:
        index_tree[parent_index] = {'parents':[], 'children':[]}
    if not child_index in index_tree:
        index_tree[child_index] = {'parents':[], 'children':[]}
    index_tree[parent_index]['children'].append(child_index)
    index_tree[child_index]['parents'].append(parent_index)

file_go_links.close()

In [13]:
# find all the terms which have no parents and are therefore the root terms
roots = []
for index in index_tree:
    if len(index_tree[index]['parents']) == 0:
        roots.append(index)

In [14]:
# computing term depths

#preparing all nodes with auxillary fields
for index in index_tree:
    index_node = index_tree[index]
    index_node['visited'] = False
    index_node['level'] = 0 # indicates unknown

#preparing root nodes
queue = roots + []
for index in queue:
    index_node = index_tree[index]
    index_node['level'] = 1
    index_node['visited'] = True

while len(queue) > 0:
    parent_index = queue.pop(0)
    parent_node = index_tree[parent_index]
    
    child_indices = parent_node['children']
    for child_index in child_indices:
        child_node = index_tree[child_index]
        if child_node['visited']:
            continue
        else:
            child_node['visited'] = True
            child_node['level'] = parent_node['level'] + 1
            queue.append(child_index)

In [15]:
max_level = 0
for index in index_tree:
    level = index_tree[index]['level']
    max_level = max(level, max_level)

In [16]:
level_counts = [0] * max_level
for index in index_tree:
    level = index_tree[index]['level']
    level_counts[level - 1] += 1

In [17]:
level_counts

[1, 22, 154, 1897, 4877, 7543, 6625, 4369, 1694, 515, 125, 31]

In [18]:
# associates each term index with all its ancestors

#prepares each tree node
for index in index_tree:
    index_node = index_tree[index]
    index_node['ancestors'] = {index}

for i in range(max_level):
    # go level by level pushing down ancestors
    cur_level = i + 1
    
    for index in index_tree:
        index_node = index_tree[index]
        
        # only look at nodes on the right level
        if index_node['level'] != cur_level:
            continue
        
        parents = index_node['parents']
        # grabs all the parents' ancestors (list of sets)
        ancestors = list(map(lambda pidx: index_tree[pidx]['ancestors'], parents))
        # unions over all parent ancestors + self (* unpacks the list of sets)
        index_node['ancestors'] = index_node['ancestors'].union(*ancestors)

In [19]:
# associates each gene index with all its ancestors
for gene_index in gene_anno:
    gene_node = gene_anno[gene_index]
    gene_node['comp'] = set().union(*list(map(lambda idx: index_tree[idx]['ancestors'], gene_node['orig'])))

In [20]:
# associates each term index with all genes indices that have it

# prepares each term index
for index in index_tree:
    index_node = index_tree[index]
    index_node['gene_inds'] = []
    
# go over each gene and append it to the indices
for gene_index in gene_anno:
    term_indices = list(gene_anno[gene_index]['comp'])
    list(map(lambda tidx: index_tree[tidx]['gene_inds'].append(gene_index), term_indices))

## Creating matchings at different levels of GO

In [37]:
# used to track the matchings we've seen on any level
total_matchings = set()

# tracks the matchings at every level (but only the highest level)
matchings = []
# can't do multiplication of lists because each entry references the same set
for i in range(max_level):
    matchings.append(set())

In [45]:
# doing this by hand instead of a loop since some levels are too big
working_level = max_level - 7

for index in index_tree:
    index_node = index_tree[index]
    
    if index_node['level'] != working_level:
        continue
    
    gene_inds = index_node['gene_inds']
    for gene_index1 in gene_inds:
        for gene_index2 in gene_inds:
            if gene_index1 == gene_index2:
                continue
            key = f'{gene_index1}-{gene_index2}'
            if not key in total_matchings:
                matchings[working_level - 1].add(key)
                total_matchings.add(key)

In [53]:
# again doing this by hand on the levels that we did
location = 'GO_links'
level = 5

file_index = open(f'{location}/level-{level}-index.txt','w')
file_gene = open(f'{location}/level-{level}-gene.txt','w')

for pair in matchings[level - 1]:
    [index1, index2] = pair.split('-')
    index1 = int(index1)
    index2 = int(index2)
    
    gene1 = go_genes[index1]
    gene2 = go_genes[index2]
    
    if gene1 in filt and gene2 in filt:
        adj1 = string_map[gene1]
        adj2 = string_map[gene2]
        file_index.write(f'{adj1}\t{adj2}\n')
        file_gene.write(f'{gene1}\t{gene2}\n')
    
file_index.close()
file_gene.close()

In [54]:
# writes everything into two big files with the level as a third value

location = 'GO_links'
min_level = 5

file_index = open(f'{location}/total-index.txt','w')
file_gene = open(f'{location}/total-gene.txt','w')

min_level = 5
for level in range(min_level - 1, max_level):
    matching = matchings[level]
    for pair in matching:
        [index1, index2] = pair.split('-')
        index1 = int(index1)
        index2 = int(index2)

        gene1 = go_genes[index1]
        gene2 = go_genes[index2]

        if gene1 in filt and gene2 in filt:
            adj1 = string_map[gene1]
            adj2 = string_map[gene2]
            file_index.write(f'{adj1}\t{adj2}\t{level+1}\n')
            file_gene.write(f'{gene1}\t{gene2}\t{level+1}\n')
        
file_index.close()
file_gene.close()

In [1]:
for i in range(10,3,-1):
    print(i)

10
9
8
7
6
5
4
